<a href="https://colab.research.google.com/github/kmalik22/colabs/blob/main/rope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import torch
import numpy as np

def create_one_rope_matrix(self, angle: float):
    return [[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]]

def create_rope_matrices(self, theta: float, max_seqlen: int, d_k: int):
    # TODO: vectorize
    d_k_by_2 = d_k//2
    all = []
    for pos in range(max_seqlen):
        pos_list = []
        for dim_by_2 in range(d_k_by_2):
            angle = pos/(theta)**(2*dim_by_2/d_k)
            mat = create_one_rope_matrix(self, angle)
            pos_list.append(mat)
        all.append(pos_list)
    return torch.tensor(all, dtype=torch.float32)


In [51]:
def t_create_rope_matrices(theta, max_seqlen, d_k):
  # create all angles: max_seq_len, d_k//2
  # create numerator and denom separately
  # numerators: (max_seqlen, d_k//2)
  numerators = torch.arange(max_seqlen).unsqueeze(1).expand(-1, d_k//2)

  # denominators: (max_seqlen, d_k//2)
  denominators = theta ** (2*torch.arange(d_k//2)/d_k).unsqueeze(0) # (1, d_k//2)
  denominators = denominators.expand(max_seqlen, -1)
  angles = numerators/denominators
  cosines = torch.cos(angles) #(max_seqlen, d_k//2)
  sines = torch.sin(angles)
  stacked = torch.stack([cosines, sines, -sines, cosines], dim=2) # (max_seqlen, d_k//2, 4)
  reshaped = stacked.reshape(max_seqlen, d_k//2, 2, 2)
  return reshaped


In [16]:
MAX_SEQLEN=2
D_K=8
THETA=100
r = create_rope_matrices(None, THETA, max_seqlen=2, d_k=8)
assert r.shape == (MAX_SEQLEN, D_K//2, 2, 2)
print(f"Shapes match")

Shapes match


In [52]:
assert(torch.allclose(t_create_rope_matrices(THETA, MAX_SEQLEN, D_K), create_rope_matrices(None, THETA, max_seqlen=2, d_k=8)))
print("Things match")

Things match
